In [1]:
import pandas as pd
import os
import re
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *

In [2]:
df = unpkl('raw_corrections.pkl')

# Known corrections

In [3]:
idxs = df.loc[(df.catalog=='97')&(df.samploc=='texture_binder5_jd')].index # unresolvable typo
df = df.drop(idxs)

In [4]:
df = df.loc[~df.fbase.str.contains('82_nn')] # unresolvable typo
df = df.loc[~df.fbase.str.contains('2078f_gb')]

In [5]:
df = df.loc[~df.catalog.str.contains('4213')] # yellow, blue, and red strips

In [6]:
df.catalog.loc[(df.catalog.str.contains('289'))&(df.samploc=='texture_box6.1_jd')] = df.catalog.loc[(df.catalog.str.contains('289'))&(df.samploc=='texture_box6.1_jd')].apply(lambda x: x.replace('9','7'))

In [7]:
df = df.loc[df.fbase!='3202a_ka_4']

In [8]:
def binder_correction(df,catalog,binder,repl):
    df.catalog.loc[(df.catalog==catalog)&(df.samploc.str.contains(binder))] = repl

In [9]:
def manbran_correction(df,catalog,manbran,repl):
    df.catalog.loc[(df.catalog==catalog)&(df.manbran==manbran)] = repl

In [10]:
binder_correction(df,'1963','binder3','1693')
manbran_correction(df,'5198','ke','5189')

In [11]:
#db = pd.read_pickle("../../db.pkl")

# Enumeration of Cases

In [12]:
df = df.loc[~df.catalog.isin(['5149','5151'])]

In [13]:
df.catalog.loc[(df.catalog=='4945')&(df.samploc=='texture_binder7_jd')] = '5554'

In [14]:
# legit re-measures or manbran typos

exact_whitelist = [
    '2198','4783s','1882c','5150','5154','4231','4949','2234','2639g','1023k','2075z','294u','5513l',
    '5528a','5529q','5612c','5494i','5490d','5615b','5618'
] 
sb_whitelist = ['5207','902','4783','4793','4885']

In [15]:
for catalog in df.catalog.unique():
    tmp = df[df.catalog==catalog]
    tmp.mtrial = tmp.mtrial.apply(int)
    tmp = tmp.sort_values('mtrial')
    
    # if all measurements are from same dir, have same manbran, and trial indices are consecutive, then ignore
    if all([len(tmp.samploc.unique())==1,len(tmp.manbran.unique())==1,is_consecutive(tmp.mtrial)]):
        continue
    if catalog in exact_whitelist: 
        continue
    barecat = ''.join([s for s in catalog if s.isdigit()])
    if barecat in sb_whitelist:
        continue
    else:
        print(catalog,end='\t')

# 5515, 5516 issue

Some cataloger made an error lettering these samples, and it has caused a lot of trouble, because many folks just ignore the letters and write what they think should come next. There is no 5515x or 5516x.

In [26]:
df = df.loc[~df.catalog.isin(['5515x','5516x'])]

In [27]:
df = df.reset_index(drop=True)

In [28]:
df.to_pickle('raw_corrections_dedupe.pkl')